### Regeneration Attacks
Regeneration attacks use off-the-shelf VAEs and diffusion models to transfer a target image $x ∈ X$ to a latent representation followed by a restoration to $x′ \in X$ that is faithful to its original representation, i.e., x≈x. Since the chosen VAE or diffusion model will not be contained by the attacker’s model of interest,
the entire regeneration is likely to disrupt the latent representation of $x$, thereby damaging an embedded watermark. However, since the capacity of the attacker’s regenerative model is inferior to the target model, $x′$ will likely be of reduced quality.

#### Prompted regeneration
We propose a simple variation on a regenerative diffusion attack: if an image is produced via a known prompt, then an attacker uses the prompt to guide the diffusion of their surrogate model.

#### Rinsing generation 2x
An image is noised then de-noised by FLUX1-dev two times with strength 0.6, guidance_scale 10, num_inference_steps=10.

We use prompt "original image" for FLUX1-dev. in our rinsing regeneration 2x attack.

In [ ]:
import torch
from diffusers import FluxImg2ImgPipeline
from huggingface_hub import login
from skimage.io import imread
import numpy as np
from PIL import Image
import os
login(LOGIN) #hugiingface login

In [ ]:
#code from https://huggingface.co/docs/diffusers/main/api/pipelines/flux

device = "cuda:2"

pipe = FluxImg2ImgPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)
pipe.to("cuda:2")

In [ ]:
def image_to_tensor(image):
    image = np.array(image).astype(np.float32) / 255.0
    image = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).to(device, dtype=torch.bfloat16)
    return image

In [ ]:
def regeneration(pipe, orig_dir, regen_dir):
    filenames = sorted([f for f in os.listdir(orig_dir) if f.endswith('.png')])
    for idx, name in enumerate(filenames):
        img1 = imread(orig_dir + "/" + name)[:,:,:3]

        img_out = pipe(
            image=image_to_tensor(img1),
            prompt="original image",
            height=512,
            width=512,
            strength=0.6,
            guidance_scale=10,
            num_inference_steps=10,
            max_sequence_length=256,
            generator=torch.Generator("cuda:1").manual_seed(42)
        ).images[0]

        img_out.save(f"{regen_dir}/{name}")

In [ ]:
def rinse2x(pipe, orig_dir, regen1x_dir, regen2x_dir):
    regeneration(pipe, orig_dir, regen1x_dir)
    regeneration(pipe, regen1x_dir, regen2x_dir)

In [ ]:
#example of usage:
#we use only images watermarked by TreeRing - (150-299 numbers)

orig_dir = "Neurips24_ETI_BeigeBox_treering"

regen1x_dir = "Neurips24_ETI_BeigeBox_treering_regen1x"
regen2x_dir = "Neurips24_ETI_BeigeBox_treering_regen2x"

rinse2x(pipe=pipe, orig_dir=orig_dir, regen1x_dir=regen1x_dir, regen2x_dir=regen2x_dir)